In [2]:
%%capture
import json
import numpy as np
import networkx as nx
import infomap
from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'infomap'

In [2]:
with open('./datasets/train_set_aan.json') as f:
    train_set = json.load(f)
    
with open('./datasets/test_set_aan.json') as f:
    test_set = json.load(f)
    
full_set = train_set + test_set

In [3]:
# create citation network
citation_graph = nx.DiGraph()
for paper in full_set:
    for ref_id in paper['references']:
        citation_graph.add_edge(paper['id'], ref_id)
        
len(citation_graph)

15602

In [38]:
# rank nodes
# don't work...

adjacency_matrix = nx.adjacency_matrix(citation_graph)

weights = (adjacency_matrix + adjacency_matrix.transpose()).sum(axis=1)

row_normalized_adjacency_matrix = adjacency_matrix / adjacency_matrix.sum(axis=1)

alef_scores = len(citation_graph) * row_normalized_adjacency_matrix.transpose() * weights / (row_normalized_adjacency_matrix.transpose() * weights).sum(axis=0)

row_normalized_adjacency_matrix.shape

/home/julien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


(15602, 15602)

In [11]:
# use classic PageRank as a substitute
pageranks = nx.pagerank(citation_graph)

In [12]:
# hierarchically cluster nodes (see https://www.mapequation.org/ and infomap package)
# source : https://github.com/mapequation/infomap/blob/master/examples/python/infomap-networkx.py

im = infomap.Infomap("--two-level")

print("Building Infomap network from a NetworkX graph...")
for source, target in citation_graph.edges:
    im.add_link(source, target)

print("Find communities with Infomap...")
im.run()

print(f"Found {im.num_top_modules} modules with codelength: {im.codelength}")

communities = im.get_modules()
nx.set_node_attributes(G, communities, 'community')

NameError: name 'infomap' is not defined

In [28]:
# recommandation selection
# source : https://github.com/jevinw/rec_utilities/tree/master/babel_util/recommenders

ground_truth = dict([(ref['id'], ref['references']) for ref in test_set])
recommendation = {}

for input_paper in tqdm(test_set):
        

len(ground_truth) - len(recommendation)

0

In [29]:
with open('./evaloffsets/stateofart_eigenfactor_gt.json', 'w') as f:
    json.dump(ground_truth, f)
    
with open('./evaloffsets/stateofart_eigenfactor_rec.json', 'w') as f:
    json.dump(recommendation, f)